In [3]:
import requests
from bs4 import BeautifulSoup
import sqlite3
import pandas as pd

database_file = "glider_tests.db"
download_folder = "data/pdf"

air_turquoise_page_size=25
air_turquise_base_url = 'https://para-test.com'
#air_turquoise_links = '{air_turquise_base_url}/component/jak2filter/?Itemid=114&issearch=1&isc=1&category_id=11&xf_3_txt=B&ordering=publishUp&orders[publishUp]=rpublishUp&orders[date]=date&start={index}'
# C : https://para-test.com/component/jak2filter/?Itemid=114&issearch=1&isc=1&category_id=11&xf_3_txt={class}&ordering=publishUp&orders[publishUp]=rpublishUp&orders[date]=date&start=0

# set up DB (once)
conn = sqlite3.connect(database_file)
cur = conn.cursor()
#cur.execute('''CREATE TABLE IF NOT EXISTS air_turquoise_reports
#               (report_date, item_name, report_link, download_link)''')
#conn.commit()
cur.execute("SELECT name FROM sqlite_master WHERE type='table';")
print(cur.fetchall())
conn.close()    
    

[('air_turquoise_reports',), ('sqlite_sequence',), ('air_turquoise_evaluation',), ('air_turquoise_parameters',), ('dhv_reports',), ('dhv_evaluation',), ('dhv_parameters',), ('test_mapping',)]


In [37]:
# add to DB  
conn = sqlite3.connect(database_file)
cur = conn.cursor()
#cur.execute("delete from air_turquoise_reports")
#cur.executemany("insert into air_turquoise_reports values (?, ?, ?, null)", data)
    
conn.commit()
conn.close() 

In [9]:
with sqlite3.connect(database_file) as conn:
    df = pd.read_sql_query("""select * from air_turquoise_reports where download_link is null order by report_link""",conn)

df.head(20)

,report_date,item_name,report_link,download_link,report_class
0,2021-05-06,Swing Flugsportgeräte Sting RS 24,/reports/item/4655-swing-flugsportgeraete-stin...,None,B
1,2021-05-06,Swing Flugsportgeräte Sting RS 26,/reports/item/4656-swing-flugsportgeraete-stin...,None,B
2,2021-05-06,Swing Flugsportgeräte Sting RS 28,/reports/item/4657-swing-flugsportgeraete-stin...,None,B
3,2021-05-06,Swing Flugsportgeräte Sting RS 31,/reports/item/4658-swing-flugsportgeraete-stin...,None,B
4,2021-05-06,Swing Flugsportgeräte Sting RS 34,/reports/item/4659-swing-flugsportgerate-sting...,None,A


In [11]:
from os.path import exists

# download all pdf
with sqlite3.connect(database_file) as conn:
    df = pd.read_sql_query("""select * from air_turquoise_reports where download_link is not null order by report_link""",conn)
    
for report in df.itertuples():
    fname = f"{download_folder}/{'_'.join(report.item_name.replace('/','').split())}.pdf"
    url = f"{air_turquise_base_url}/{report.download_link}" if report.download_link.startswith('storage') else f"{air_turquise_base_url}{report.download_link}" 
    if not exists(fname):
        print(fname,url)
        #r = requests.get(url, allow_redirects=True)
        #open(fname, 'wb').write(r.content) 

df.tail()



data/pdf/Niviuk_Gliders_Kode_P_18_(extended_"medium"_weight_range).pdf https://para-test.com/reports/item/download/32096_4fec4fab051d840d82dcbb3308390766


,report_date,item_name,report_link,download_link,report_class
0,2020-01-23,Windtech Paragliders Duster XS,/reports/item/4220-windtech-paragliders-duster-xs,/reports/item/download/28996_194211aac538e2e6f...,B
1,2020-01-23,Windtech Paragliders Duster S,/reports/item/4221-windtech-paragliders-duster-s,/reports/item/download/28221_c67de0f1228b0ef1e...,B
2,2020-01-23,Windtech Paragliders Duster M,/reports/item/4222-windtech-paragliders-duster-m,/reports/item/download/28227_0c46a9b5150e856e6...,B
3,2020-01-23,Windtech Paragliders Duster L,/reports/item/4223-windtech-paragliders-duster-l,/reports/item/download/28233_4c0f212b723ca7afa...,B
4,2019-10-10,Swing Flugsportgeräte Helios RS S,/reports/item/4273-swing-flugsportgeraete-heli...,/reports/item/download/28514_564bea24900767948...,C
